<a href="https://colab.research.google.com/github/VibhaeoM01/ML-/blob/main/FakeNewsclassifierusingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all we clean this data by:
removing NA,then splitting dependent and independent variables
importing few lib- embedding, pad_sequences,LSTM,sequential, ONE HOT, DENSE,
then we preprocess it by removing stopwords, embedding representation and then we create model

In [1]:
import pandas as pd

In [19]:
df=pd.read_csv('train.csv')

In [20]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
df=df.dropna()

In [22]:
#get independent features
X=df.drop('label',axis=1)

In [23]:
X.shape

(18285, 4)

In [24]:
y=df.label

In [25]:
y.shape

(18285,)

In [26]:
import tensorflow as tf

In [27]:
tf.__version__

'2.15.0'

In [28]:
 from tensorflow.keras.layers import Embedding
 from tensorflow.keras.preprocessing.sequence import pad_sequences   # this is used to keep the length of sentences same , so we add padding either in front or in back
 from tensorflow.keras.models import Sequential
 from tensorflow.keras.preprocessing.text import one_hot  #used to convert sentences into one hot representation,given a vocab size
 from tensorflow.keras.layers import LSTM
 from tensorflow.keras.layers import Dense #Because as it is a classification problem there should be last final 1 layer for output that it is >0.5 or <0.5

In [29]:
voc_size=5000

Text preprocessing

In [30]:
messages=X.copy()
messages.reset_index(inplace=True)  #resetbecause we have frop NAN values.

In [34]:
import nltk
import re
from nltk.corpus import stopwords

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
##data preprocessing
from nltk.stem import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])  # sub=substitute
  review=review.lower()
  review=review.split()
  review=[ps.stem(i) for i in review if i not in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)


In [39]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

#One hot

In [41]:
onehot_repr=[one_hot(i,voc_size) for i in corpus]
onehot_repr

[[4422, 2007, 2993, 1169, 1668, 2105, 4023, 2211, 3951, 2564],
 [2860, 578, 3264, 958, 903, 3939, 597],
 [3449, 863, 1168, 2494],
 [4724, 3852, 1709, 2548, 1307, 4450],
 [4115, 903, 66, 1397, 3966, 3963, 903, 3856, 1134, 1952],
 [495,
  2814,
  2491,
  2525,
  3012,
  2637,
  4422,
  2186,
  1283,
  1348,
  1700,
  4881,
  1320,
  3531,
  597],
 [3178, 4757, 2771, 1132, 3100, 75, 1215, 4211, 2352, 4038, 870],
 [1187, 181, 1327, 3225, 2227, 3209, 2637, 4037, 2352, 4038, 870],
 [2079, 2769, 3638, 4708, 4592, 3950, 1041, 2304, 2637, 2866],
 [2043, 3548, 174, 4797, 2873, 3809, 2433, 2603],
 [1540, 3530, 4476, 3476, 1779, 3084, 1244, 3506, 3546, 4954, 909],
 [2548, 2268, 1668, 3950, 2637, 2227],
 [4189, 1754, 3464, 2062, 4426, 976, 472, 2661, 1136],
 [3798, 357, 1793, 2077, 4194, 4129, 2103, 2352, 4038, 870],
 [4386, 2796, 446, 405, 2001, 2352, 4038, 870],
 [3985, 986, 1917, 1949, 608, 2910, 2962, 3634, 695, 4292],
 [1598, 1497, 578],
 [1523, 2382, 1249, 1866, 2637, 1682, 1539, 597],
 [4686

#Embedding representation

In [46]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

#padding is applied here.

[[   0    0    0 ... 2211 3951 2564]
 [   0    0    0 ...  903 3939  597]
 [   0    0    0 ...  863 1168 2494]
 ...
 [   0    0    0 ... 2352 4038  870]
 [   0    0    0 ... 3652 1178  705]
 [   0    0    0 ... 3522 2338 1182]]


In [47]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4422,
       2007, 2993, 1169, 1668, 2105, 4023, 2211, 3951, 2564], dtype=int32)

In [66]:
from tensorflow.keras.layers import Dropout
#Creating the model
embedding_vector_features=40
model=Sequential() #
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #embedding layer is added ,1st para=voc_size,2ndpara=features size,,3rd para=sentence length
model.add(LSTM(100))  #passing 1 LSTM layer having 100 neurons
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid')) #as it is classification problem we add dense layer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [67]:
len(embedded_docs), y.shape

(18285, (18285,))

In [68]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [69]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [70]:
#splitting into train test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

##Here we train the model

In [71]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 44ms/step - loss: 0.3377 - accuracy: 0.8365 - val_loss: 0.1949 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1433 - accuracy: 0.9438 - val_loss: 0.1902 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1000 - accuracy: 0.9647 - val_loss: 0.2070 - val_accuracy: 0.9220
Epoch 4/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0681 - accuracy: 0.9768 - val_loss: 0.2452 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0463 - accuracy: 0.9845 - val_loss: 0.2817 - val_accuracy: 0.9180
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0267 - accuracy: 0.9924 - val_loss: 0.3506 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0158 - accuracy: 0.9960 - val_loss: 0.4555 - val_accuracy: 0.

#Performance Metrics and Accuracy

In [72]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

189/189 [==============================] - 2s 9ms/step


In [73]:
from sklearn.metrics import confusion_matrix

In [74]:
confusion_matrix(y_test,y_pred)

array([[3052,  367],
       [ 160, 2456]])

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9126760563380282